In [11]:
# -----------------------------------------------------------
# handling arithmetic operations at bitwise level
# -----------------------------------------------------------
#
# implementation isn't perfect and therefore is not
# recommended for use in projects, but can provide
# some clues about how do bitwise operations
# work and be useful for study purposes
#
# current implementation can (and do) contain some
# misleading parts about arithmetic logic at bitwise
# level, so learn with care
#
# at the moment of publishing this code I found
# much simpler and more efficient ways of
# implementing this to provide more clear and
# understandable code, but I decided to post
# this anyway
#
# -----------------------------------------------------------
# Released under MIT Public License (MIT)
# (C) Alexander Isaychikov, Minsk, Belarus
# email isaychikov.ai@yahoo.com
# -----------------------------------------------------------

In [12]:
import math
import warnings

In [13]:
class Bitset:
    """
    The Bitset class represents a sequence of bits and implements all bitwise operations

    Parameters
    ----------
    length : int
        Indicates length of bit sequence
    value : int/list/Bitset
        Value for converting into binary format, or container to copy binary sequence from
    sign_bit : bool
        Indicates whether the last bit should be used as sign bit or not

    Variables
    ---------
    length : int
        This is where we store length
    sign_bit : bool
        This is where we store sign bit usage flag
    container : list
        This is where we store sequence of bits
    """

    def __init__(self, length, value, sign_bit=True):

        self.length = length
        self.sign_bit = sign_bit

        if type(value) is type(int()):
            value = self.to_binary(value)
        elif type(value) is not type(list()):
            value = value.container
        self.container = value

    def to_binary(self, decimal):
        """
        Function to convert decimal into bit sequence

        Parameters
        ----------
        decimal : int
            Value in decimal format

        Returns
        -------
        list
            Value in binary format
        """

        overflow_check(self.length, decimal, self.sign_bit)

        output = [0] * self.length

        if self.sign_bit:
            length = self.length - 1
            output[length] = 0 if (decimal >= 0) else 1
            decimal = abs(decimal)
        else:
            length = self.length

        for i in range(length):
            output[i] = 1 if (int(decimal % 2) == 1) else 0
            decimal /= 2

        return output

    def to_decimal(self):
        """
        Function to convert binary into decimal

        Returns
        -------
        int
            Value in decimal format
        """

        length = self.length - 1 if (self.sign_bit == True) else self.length

        output = 0
        for i in range(length):
            output += self[i] * pow(2, i)
        return -output if (self.sign_bit is True and self[length] == 1) else output

    def _binary(self):
        s = ""
        for i, bit in enumerate(self.container):
            s = ("1" if bit == 1 else "0") + s
            if i < self.length - 1 and (i + 1) % 4 == 0:
                s = ' ' + s
        return s

    def __and__(self, other):
        other = Bitset._cast_to_bitset(other, self)

        length_equality_check(self, other)

        result = [0] * self.length
        for i in range(self.length):
            result[i] = 1 if (self[i] & other[i]) else 0

        return Bitset(self.length, result, sign_bit=self.sign_bit)

    def __or__(self, other):
        other = Bitset._cast_to_bitset(other, self)

        length_equality_check(self, other)

        result = [0] * self.length
        for i in range(self.length):
            result[i] = 1 if (self[i] | other[i]) else 0

        return Bitset(self.length, result, sign_bit=self.sign_bit)

    def __invert__(self):
        result = [0] * self.length
        for i in range(self.length):
            result[i] = 1 if (self[i] == 0) else 0

        return Bitset(self.length, result, sign_bit=self.sign_bit)

    def __xor__(self, other):
        other = Bitset._cast_to_bitset(other, self)

        length_equality_check(self, other)

        result = [0] * self.length
        for i in range(self.length):
            result[i] = 1 if (self[i] ^ other[i]) else 0

        return Bitset(self.length, result, sign_bit=self.sign_bit)

    def __lshift__(self, value):
        if value < 0:
            raise Exception("value must be positive")

        result = [0] * self.length
        result[value:self.length] = self[:(self.length - value)]

        return Bitset(self.length, result, sign_bit=self.sign_bit)

    def __rshift__(self, value):
        if value < 0:
            raise Exception("value must be positive")

        result = [0] * self.length
        result[:(self.length - value)] = self[value:self.length]

        return Bitset(self.length, result, sign_bit=self.sign_bit)

    def __eq__(self, other):
        other = Bitset._cast_to_bitset(other, self)
        return self.container == other.container

    def __ne__(self, other):
        other = Bitset._cast_to_bitset(other, self)
        return self.container != other.container

    def __lt__(self, other):
        other = Bitset._cast_to_bitset(other, self)
        return self.to_decimal() < other.to_decimal()

    def __gt__(self, other):
        other = Bitset._cast_to_bitset(other, self)
        return self.to_decimal() > other.to_decimal()

    def __le__(self, other):
        other = Bitset._cast_to_bitset(other, self)
        return self.to_decimal() <= other.to_decimal()

    def __ge__(self, other):
        other = Bitset._cast_to_bitset(other, self)
        return self.to_decimal() >= other.to_decimal()

    def __int__(self):
        return self.to_decimal()

    def __str__(self):
        return self._binary()

    def __repr__(self):
        return f"Bitset({str(self)})"

    def __getitem__(self, s):
        return self.container[s]

    def __setitem__(self, s, value):
        self.container[s] = value
        return self

    def __iter__(self):
        for i in self[:]:
            yield i

    def __len__(self):
        return self.length

    def _cast_to_bitset(value, example):
        if type(value) in [type(int()), type(list())]:
            return Bitset(example.length, value, example.sign_bit)
        return value


def overflow_check(length, decimal, sign_bit):
    length = length - 1 if sign_bit is True else length
    sign = -1 if sign_bit else 0
    if not sign * pow(2, length) - sign <= decimal <= pow(2, length) - 1:
        warnings.warn(f"\n Decimal {decimal} doesn't pass in the" + \
                      (" signed" if sign_bit is True else "") + \
                      f" bitset with range of " + \
                      f"[{sign * pow(2, length) - sign};{pow(2, length) - 1}] \n")


def length_equality_check(left, right):
    if left.length != right.length:
        raise Exception("length of both bitset variables must be equal")
    if left.sign_bit != right.sign_bit:
        raise Exception("both bitsets must be either signed or unsigned")

In [14]:
class int32(Bitset):
    """
    The int32 class inherits Bitset class and stores signed bitset
    with length of 32 bits, implementing standard arithmetic operations

    Parameters
    ----------
    value : int/list/Bitset
        Value for converting into binary format, or container to copy binary sequence from
    """

    def __init__(self, value):
        super().__init__(32, value, sign_bit=True)

    def __add__(self, other):
        return int32.summation(self, other)

    def __sub__(self, other):
        return int32.summation(self, -other)

    def __neg__(self):
        return self ^ (int32(1) << 31)

    def __mul__(self, other):
        return int32.mul(self, other)

    def __truediv__(self, other):
        return int32.div(self, other)

    def __str__(self):
        return str(self.to_decimal())

    def __repr__(self):
        return f"int32({self._binary()} = {str(self)})"

    def _binary(self):
        return super()._binary()

    @staticmethod
    def sum_signed(left, right):

        x1 = left
        x2 = right
        shift = int32(0)

        while x2 != 0:
            shift = x1 & x2
            x1 = x1 ^ x2
            x2 = shift << 1

        return int32(x1)

    @staticmethod
    def sum_inverse(left, right):

        x1 = left
        x2 = right
        sign_bit = len(left) - 1
        overflow = int32(0)
        shift = int32(0)

        if x1[sign_bit] == 1:
            x1 = ~x1
            x1[sign_bit] = 1

        if x2[sign_bit] == 1:
            x2 = ~x2
            x2[sign_bit] = 1

        overflow = int32._get_overflow(x1, x2)

        while x2 != 0:
            shift = x1 & x2
            x1 = x1 ^ x2
            x2 = shift << 1

        x1 = int32.sum_signed(x1, overflow)

        if x1[sign_bit] == 1:
            x1 = ~x1
            x1[sign_bit] = 1

        return int32(x1)

    @staticmethod
    def sum_complimentary(left, right):

        x1 = int32(left)
        x2 = int32(right)
        sign_bit = len(left) - 1
        shift = int32(0)

        if x1[sign_bit] == 1:
            x1 = ~x1
            x1[sign_bit] = 1
            x1 = int32.sum_signed(x1, 1)

        if x2[sign_bit] == 1:
            x2 = ~x2
            x2[sign_bit] = 1
            x2 = int32.sum_signed(x2, 1)

        while x2 != 0:
            shift = x1 & x2
            x1 = x1 ^ x2
            x2 = shift << 1

        if x1[sign_bit] == 1:
            x1 = ~x1
            x1[sign_bit] = 1
            x1 = int32.sum_signed(x1, 1)

        return int32(x1)

    @staticmethod
    def summation(left, right):
        return int32(int32.sum_complimentary(left, right))

    @staticmethod
    def mul(left, right):

        x1 = left
        x2 = right
        result = int32(0)
        sign = int32(0)

        sign = ((x1 ^ x2) >> 31) << 31
        x1[31] = 0
        x2[31] = 0

        while x2 != 0:
            if x2[0] == 1:
                result = int32.sum_complimentary(result, x1)

            x1 = x1 << 1
            x2 = x2 >> 1

        result = result | sign

        return int32(result)

    @staticmethod
    def div(left, right):

        x1 = int32(left)
        x2 = int32(right)
        counter = int32(1)
        quotient = int32(0)
        sign = ((x1 ^ x2) >> 31) << 31

        x1[31] = 0
        x2[31] = 0

        while x2 <= x1:
            x2 = x2 << 1
            counter = counter << 1

        while counter > 1:
            x2 = x2 >> 1
            counter = counter >> 1
            if x1 >= x2:
                x1 = int32(x1) - int32(x2)
                quotient = int32.sum_complimentary(quotient, counter)

        quotient = quotient | sign

        return int32(quotient)

    def _get_overflow(left, right):

        last_bit = len(left) - 1
        x1 = left
        x2 = right
        shift = int32(0)

        while x2 != int32(0):
            shift = x1 & x2
            if shift[last_bit] == 1:
                return 1

            x1 = x1 ^ x2
            x2 = shift << 1

        return 0

In [15]:
class float32(int32):
    """
    The float32 class inherits int32 class and stores signed bitset
    with length of 32 bits, implementing sum operation following IEEE754 standart

    Parameters
    ----------
    value : int/list/Bitset
        Value for converting into binary format, or container to copy binary sequence from
    """

    def __init__(self, value):
        if type(value) in [type(int()), type(float())]:
            super().__init__(self.to_binary(value))
        else:
            super().__init__(value)

    def __add__(self, other):
        return float32.float_sum(self, other)

    def __str__(self):
        return str(self.to_decimal())

    def __repr__(self):
        return f"float32({self._binary()} = {str(self)})"

    def _binary(self):
        return super()._binary()

    def to_binary(self, decimal):
        out = int32(127)

        if decimal == 0:
            return 0

        while decimal >= 2:
            decimal /= 2
            out = out + 1

        while decimal < 1:
            decimal *= 2
            out = out - 1

        out = out << 23

        decimal -= 1

        for i in range(23):
            if pow(2, -i - 1) <= decimal:
                decimal -= pow(2, -i - 1)
                out[22 - i] = 1

        return out

    def to_decimal(self):
        output = 1
        exponent = (self >> 23).to_decimal() - 127

        exponent = pow(2, exponent)
        for i in range(23):
            if self[i] == 1:
                output += pow(2, i - 23)

        return output * exponent

    @staticmethod
    def float_sum(left, right):

        x1 = left
        x2 = right
        output = float32(0)
        bias = (x2 >> 23).to_decimal() - (x1 >> 23).to_decimal()

        mantis_x1 = (x1 << 9) >> 9
        mantis_x2 = (x2 << 9) >> 9
        mantis_x2[23] = 1
        mantis_x1[23] = 1

        if bias > 0:
            exponent = (x2 >> 23) << 23
            mantis_x1 = mantis_x1 >> bias
        elif bias < 0:
            bias = abs(bias)
            exponent = (x1 >> 23) << 23
            mantis_x2 = mantis_x2 >> bias
        else:
            exponent = (x1 >> 23) << 23

        output = int32(mantis_x1) + int32(mantis_x2)
        if output[23] == 0:
            output = output >> 1
            exponent = int32(exponent) + int32(int32(1) << 23)
        output[23] = 0

        output = output | exponent

        return float32(output)

# Examples

In [16]:
x = 4
y = 19

In [17]:
for a, b in zip([int32(x), int32(-x), int32(x), int32(-x)], [int32(y), int32(y), int32(-y), int32(-y)]):
    print(f'[SIGNED]\t {a} + {b} = {int32.sum_signed(a, b)}')
    print(f'[INVERSE]\t {a} + {b} = {int32.sum_inverse(a, b)}')    
    print(f'[COMPLIMENTARY]\t {a} + {b} = {int32.sum_complimentary(a, b)}')        

[SIGNED]	 4 + 19 = 23
[INVERSE]	 4 + 19 = 23
[COMPLIMENTARY]	 4 + 19 = 23
[SIGNED]	 -4 + 19 = -23
[INVERSE]	 -4 + 19 = 15
[COMPLIMENTARY]	 -4 + 19 = 15
[SIGNED]	 4 + -19 = -23
[INVERSE]	 4 + -19 = -15
[COMPLIMENTARY]	 4 + -19 = -15
[SIGNED]	 -4 + -19 = 23
[INVERSE]	 -4 + -19 = -23
[COMPLIMENTARY]	 -4 + -19 = -23


In [18]:
for a, b in zip([int32(x), int32(-x), int32(x), int32(-x)], [int32(y), int32(y), int32(-y), int32(-y)]):
    print(f'[MUL]\t {a} * {b} = {a * b}')    
    print(f'[DIV]\t {a} / {b} = {a / b}')    

[MUL]	 4 * 19 = 76
[DIV]	 4 / 19 = 0
[MUL]	 -4 * 19 = -76
[DIV]	 4 / 19 = 0
[MUL]	 4 * -19 = -76
[DIV]	 4 / 19 = 0
[MUL]	 -4 * -19 = 76
[DIV]	 4 / 19 = 0


In [19]:
x = 0.5
y = 0.235

In [20]:
for a, b in zip([float32(x)], [float32(y)]):
    print(f'[SUM]\t {a} + {b} = {a + b}')    

[SUM]	 0.5 + 0.23499999940395355 = 0.73499995470047
